In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(dotenv_path = find_dotenv('/home/jlh5498/transition_auger/secrets.env'), override=True)

In [ ]:
import psycopg

# Connect to an existing database
with psycopg.connect(f"dbname=berkeley user=james host=chaoss.tv port=5432 password='{os.getenv('AUGUR_PASSWORD')}'") as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cursor:

        # Execute a command: this creates a new table
        cursor.execute("""
            SELECT * from information_schema.columns 
            ORDER BY table_schema, table_name
            """)
        for row in cursor:
            print(row)